In [1]:
# !apt update > /dev/null; apt install aria2 git-lfs axel -y > /dev/null
# !pip install ltralytics==8.2.0 numpy pandas opencv-python Pillow matplotlib > /dev/null
# !axel -n 12 -a http://mirror.coggle.club/seg_risky_testing_data.zip; unzip -q seg_risky_testing_data.zip
# !axel -n 12 -a  http://mirror.coggle.club/seg_risky_training_data_00.zip; unzip -q seg_risky_training_data_00.zip

In [2]:
# !pip install ultralytics==8.2.0 numpy pandas opencv-python Pillow matplotlib > /dev/null
# !axel -n 12 -a http://mirror.coggle.club/seg_risky_testing_data.zip -o ~/data/seg_risky/seg_risky_testing_data.zip
# !unzip -q /home/tipriest/data/seg_risky/seg_risky_testing_data.zip -d /home/tipriest/data/seg_risky/

In [3]:
# !axel -n 12 -a  http://mirror.coggle.club/seg_risky_training_data_00.zip -o ~/data/seg_risky/seg_risky_training_data_00.zip
# !unzip -q /home/tipriest/data/seg_risky/seg_risky_training_data_00.zip -d /home/tipriest/data/seg_risky/

In [4]:
data_path = "/home/tipriest/data/seg_risky"

In [5]:
import os, shutil
import cv2
import glob
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# FIXME: 这里使用的数据是100万数据中的6万张
training_anno = pd.read_csv('http://mirror.coggle.club/seg_risky_training_anno.csv')

train_jpgs = [x.replace('/home/tipriest/data/seg_risky/', '') for x in glob.glob(os.path.join(data_path, '0/*.jpg'))]
training_anno = training_anno[training_anno['Path'].isin(train_jpgs)]
training_anno['Polygons'] = training_anno['Polygons'].apply(json.loads)

In [6]:
print("trainging_anno.shape = ", training_anno.shape)
print(np.array(training_anno['Polygons'].iloc[4], dtype=np.int32))
print(training_anno.info())
training_anno.head()

trainging_anno.shape =  (63785, 2)
[[[393 398]
  [442 398]
  [442 420]
  [393 420]]

 [[393 334]
  [467 334]
  [467 356]
  [393 356]]

 [[  4 291]
  [100 291]
  [100 312]
  [  4 312]]]
<class 'pandas.core.frame.DataFrame'>
Index: 63785 entries, 13 to 1020545
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Path      63785 non-null  object
 1   Polygons  63785 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB
None


,Path,Polygons
13,0/0a3907dbf1ce7cd699020e0df8492920.jpg,"[[[240, 403], [153, 397], [154, 346], [241, 35..."
17,0/0d3898109a2f973bd18f9976a8669d77.jpg,"[[[166, 69], [322, 62], [323, 72], [485, 62], ..."
21,0/0dab23eb6a6359765bdf8a7570e7e83b.jpg,"[[[174, 321], [174, 212], [331, 212], [331, 32..."
42,0/02e87d696c089ef3bb98e6927f346d30.jpg,"[[[126, 325], [126, 304], [156, 304], [156, 32..."
51,0/0006b88e2b714a9d38c022e49cec28e4.jpg,"[[[393.0, 398.0], [442.0, 398.0], [442.0, 420...."


## Format dateset

In [7]:
yolo_seg_dataset_path = os.path.join(data_path, "dataset")
yolo_seg_dataset_train_path = os.path.join(yolo_seg_dataset_path, "train/")
yolo_seg_dataset_valid_path = os.path.join(yolo_seg_dataset_path, "valid/")

if os.path.exists(yolo_seg_dataset_path):
    shutil.rmtree(yolo_seg_dataset_path)
os.makedirs(yolo_seg_dataset_train_path)
os.makedirs(yolo_seg_dataset_valid_path)

def normalize_polygon(polygon, img_width, img_height):
    return [(x / img_width, y / img_height) for x, y in polygon]

for row in training_anno.iloc[10000:].iterrows():
    shutil.copy(os.path.join(data_path, row[1].Path), yolo_seg_dataset_train_path)
    img = cv2.imread(os.path.join(data_path, row[1].Path))
    img_height, img_width = img.shape[:2]
    txt_filename = os.path.join(yolo_seg_dataset_train_path + row[1].Path.split('/')[-1][:-4] + '.txt')
    with open(txt_filename, 'w') as up:
        for polygon in row[1].Polygons:
            normalized_polygon = normalize_polygon(polygon, img_width, img_height)
            normalized_coords = ' '.join([f'{coord[0]:.3f} {coord[1]:.3f}' for coord in normalized_polygon])
            up.write(f'0 {normalized_coords}\n')

In [8]:
# 制作验证集
for row in training_anno.iloc[:10000].iterrows():
    shutil.copy(os.path.join(data_path, row[1].Path), yolo_seg_dataset_valid_path)

    img = cv2.imread(os.path.join(data_path, row[1].Path))
    img_height, img_width = img.shape[:2]
    txt_filename = os.path.join(yolo_seg_dataset_valid_path + row[1].Path.split('/')[-1][:-4] + '.txt')
    with open(txt_filename, 'w') as up:
        for polygon in row[1].Polygons:
            normalized_polygon = normalize_polygon(polygon, img_width, img_height)
            normalized_coords = ' '.join([f'{coord[0]:.3f} {coord[1]:.3f}' for coord in normalized_polygon])
            up.write(f'0 {normalized_coords}\n')

In [9]:
with open(os.path.join(yolo_seg_dataset_path, "data.yaml"), 'w') as up:
    data_root = os.path.abspath(yolo_seg_dataset_path)
    up.write(f'''
path: {data_root}
train: train
val: valid

names:
    0: alter
''')

## Load Pretrained Model & Train model

In [10]:
# !wget http://mirror.coggle.club/yolo/Arial.ttf -O /home/tipriest/.cache/torch/hub/checkpoints/Arial.ttf
# !wget http://mirror.coggle.club/yolo/yolov8n-v8.2.0.pt -O /home/tipriest/.cache/torch/hub/checkpoints/yolov8n.pt
# !wget http://mirror.coggle.club/yolo/yolov8n-seg-v8.2.0.pt -O /home/tipriest/.cache/torch/hub/checkpoints/yolov8n-seg.pt

In [11]:
from ultralytics import YOLO
model_base_path = "/home/tipriest/.cache/torch/hub/checkpoints"
# yolov8n-seg、yolov8s-seg、yolov8m-seg、yolov8l-seg和yolov8x-seg，模型的大小依次递增
model = YOLO(os.path.join(model_base_path, "yolov8m-seg.pt"))
results = model.train(data=os.path.join(yolo_seg_dataset_path, "data.yaml"), epochs=30, imgsz=512)

/home/tipriest/anaconda3/envs/AIdefense/lib/python3.10/site-packages/ultralytics/nn/tasks.py:732: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_l

New https://pypi.org/project/ultralytics/8.3.12 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.0 🚀 Python-3.10.14 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 15959MiB)
engine/trainer: task=segment, mode=train, model=/home/tipriest/.cache/torch/hub/checkpoints/yolov8m-seg.pt, data=/home/tipriest/data/seg_risky/dataset/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=F

/home/tipriest/anaconda3/envs/AIdefense/lib/python3.10/site-packages/ultralytics/nn/tasks.py:732: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_l

AMP: checks passed ✅


/home/tipriest/anaconda3/envs/AIdefense/lib/python3.10/site-packages/ultralytics/utils/checks.py:640: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):
/home/tipriest/anaconda3/envs/AIdefense/lib/python3.10/site-packages/ultralytics/engine/trainer.py:261: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning /home/tipriest/data/seg_risky/dataset/train... 53785 images, 0 backgrounds, 18 corrupt: 100%|██████████| 53785/53785 [00:21<00:00, 2509.88it/s]

...


      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      30/30      5.14G     0.7894     0.8673     0.5458     0.9577         12        512: 100%|██████████| 3361/3361 [06:59<00:00,  8.01it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [00:39<00:00,  7.96it/s]


                   all       9998      14680      0.952      0.858      0.927      0.764      0.949      0.855      0.922      0.626

30 epochs completed in 3.817 hours.
Optimizer stripped from runs/segment/train5/weights/last.pt, 54.8MB
Optimizer stripped from runs/segment/train5/weights/best.pt, 54.8MB

Validating runs/segment/train5/weights/best.pt...
Ultralytics YOLOv8.2.0 🚀 Python-3.10.14 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 15959MiB)
YOLOv8m-seg summary (fused): 245 layers, 27222963 parameters, 0 gradients, 110.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [00:35<00:00,  8.74it/s]


                   all       9998      14680      0.951      0.859      0.927      0.764      0.948      0.855      0.922      0.627
Speed: 0.1ms preprocess, 1.7ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/segment/train5


In [13]:
from ultralytics import YOLO
import glob
from tqdm import tqdm
import os

model = YOLO("./runs/segment/train5/weights/best.pt")

test_imgs = glob.glob(os.path.join(data_path, "test_set_A_rename/*/*"))
# test_imgs = glob.glob(os.path.join(data_path, "data_set/train/0/*"))
# print(os.path.join(data_path, "data_set/train/0/*"))

In [26]:
import numpy as np
Polygon = []
for path in tqdm(test_imgs[:]):
    # print(path)
    results = model(path, verbose=False)
    result = results[0]
    if result.masks is None:
        Polygon.append([])
    else:
        image_polygons = []
        for mask in result.masks.xy:
            # 将掩膜的坐标转换为Numpy数组
            mask_coords = np.array(mask)
            # 获取外接矩形(bounding box)
            # print(mask_coords)
            # print()
            if 0 == mask_coords.size:
                continue
            x_min = round(float(np.min(mask_coords[:, 0])), 1)
            x_max = round(float(np.max(mask_coords[:, 0])), 1)
            y_min = round(float(np.min(mask_coords[:, 1])), 1)
            y_max = round(float(np.max(mask_coords[:, 1])), 1)
            bounding_box = [
                [x_min, y_min],
                [x_min, y_max],
                [x_max, y_max],
                [x_max, y_min]
            ]
            image_polygons.append(bounding_box)
            # print(bounding_box)
        Polygon.append(image_polygons)
        

100%|██████████| 100000/100000 [10:52<00:00, 153.24it/s]


In [27]:
import pandas as pd
submit = pd.DataFrame({
    'Path': [x.split('/')[-1] for x in test_imgs[:]],
    'Polygon': Polygon
})

In [28]:
submit = pd.merge(submit, pd.DataFrame({'Path': [x.split('/')[-1] for x in test_imgs[:]]}), on='Path', how='right')
submit = submit.fillna('[]')
submit.to_csv('track2_submit.csv', index=None)